## Accelerating Metal-Organic Framework Discovery via Synthesisability Prediction: The MFD Evaluation Method for One-Class Classification Models

Chi Zhang, Dmytro Antypov, Matthew J. Rosseinsky, and Matthew S. Dyer*<br/>
Email: M.S.Dyer@liverpool.ac.uk <br/>

Please cite the corresponding paper if you used the Maximum Fractional Differe (MFD) method or these trained machine learning (ML) models in your work.<br/>

The Jupyter Notebook hereafter presents the generation of the ground-truth and query dataset, along with the featurization process (as shown in Figure 1, step 1-3). The ground-truth dataset, containing 3D-connected MOF networks made of a single metal and a single linker species in the CSD MOF subset, is obtained from Pétuya et al.’s 1M1L3D dataset [DOI: 10.1002/anie.202114573] used for predicting the guest accessibility of MOFs. Taking all of the separate metals and linkers contained in the 1M1L3D dataset, we then generate a larger query dataset of 160,582 potential MOF chemistries by considering every pairwise combination of these metals and linkers that are not contained in the 1M1L3D dataset.

Three different featurization approaches are used for both metals and linkers. This notebook details the featurization of metals using 6 elemental features, magpie, and mat2vec, and the featurization of linkers using Mordred and RDKit. The process for generating ECFPs is shown in the 'one_class_classification_models' folder.

This notebook requires the following two input files related to 1M1L3D dataset. These two documents are publicly available from (http://datacat.liverpool.ac.uk/1494):

`1M1L3D_summary.csv` - the list of metal and linker constituents of 14,296 reported 3D MOF structures<br/>
`1M1L3D_metal_descriptors.csv` - the list of 6 metal features<br/>

This notebook outputs the following files:

`ground_truth_pairs.csv` - the list of metal and linker combinations of the ground_truth dataset<br/>
`query_pairs.csv` - the list of metal and linker combinations of the query dataset<br/>
`linker_list.pkl` - the list of linkers<br/>
`metal_scaled_6.csv` - the list of 6 metal features<br/>
`metal_scaled_27.csv` - the list of 27 metal features<br/>
`metal_scaled_205.csv` - the list of 205 metal features<br/>
`linker_scaled_1613.csv` - the list of 1613 mordred linker features<br/>
`linker_scaled_177.csv` - the list of 177 mordred linker features after removing feature columns with Pearson correlation greater than 0.80 and low variance below 0.50<br/>
`linker_scaled_rdkit.csv` - the list of 194 RDkit linker features<br/>

WARNING:
In order to ensure that Mordred descriptors are properly calculated user must use the version 2.1.0 of package `networkx`. It can be necessary to downgrade it as follow:<br/>
`conda install -c anaconda networkx=2.1.0`

In [1]:
#import libraries

import networkx
print(f'networkx: {networkx.__version__}')

import numpy as np
from numpy import nan as NaN
print(f'numpy: {np.__version__}')


import rdkit
from rdkit import Chem, DataStructs
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole
from rdkit.ML.Descriptors import MoleculeDescriptors
print(f'rdkit: {rdkit.__version__}')

import mordred
from mordred import Calculator, descriptors
print(f'mordred: {mordred.__version__}')

import pandas as pd
print(f'pandas: {pd.__version__}')

import numpy as np
from numpy import nan as NaN

import sklearn
from sklearn.feature_selection import f_classif, SelectKBest, VarianceThreshold
from sklearn.preprocessing import StandardScaler
print(f'sklearn: {sklearn.__version__}')

networkx: 2.1
numpy: 1.21.5
rdkit: 2021.03.5
mordred: 1.2.0
pandas: 1.2.5
sklearn: 0.24.2


# Dataset Generation

In [3]:
#import the ground_truth dataset
labelled = pd.read_csv('./1M1L3D_summary.csv')
labelled.drop(['refcode','doi','Largest Cavity Diameter','Pore Limiting Diameter','Largest Free Sphere'], axis=1, inplace=True)
labelled = labelled.loc[:,['metal','linker SMILES']]
labelled['metal'] = labelled['metal'].str.strip()

#convert linker SMILES strings to canonical SMILES strings
for index,row in labelled.iterrows():    
    mol = Chem.MolFromSmiles(row['linker SMILES'])
    row['linker SMILES'] = Chem.MolToSmiles(mol)

#drop duplicates
labelled.drop_duplicates(inplace=True)

#the ground_truth dataset
print(labelled)

      metal                                      linker SMILES
0        Co                                     O=C(O)c1ccncc1
3        Ag                               O=C(O)c1nccnc1C(=O)O
4        Mn                     O=C(O)c1cc(C(=O)O)cc(C(=O)O)c1
7        Co                     O=C(O)c1cc(C(=O)O)cc(C(=O)O)c1
8        Tb  OS(O)(O)c1ccc(-c2c3nc(c(-c4ccc(S(O)(O)O)cc4)c4...
...     ...                                                ...
14270    Rb  O=C1C([N+](=O)[O-])=CC([N+](=O)[O-])=C[C@@H]1[...
14271    Rb  O=C1C([N+](=O)[O-])=CC([N+](=O)[O-])C=C1[N+](=...
14282     K                       O=C1C(=O)C(=O)C(=O)C(=O)C1=O
14283    Rb                         O=C(O)CC(O)(CC(=O)O)C(=O)O
14284    Cs  O=[N+]([O-])c1cc([N+](=O)[O-])c(Nc2c([N+](=O)[...

[7375 rows x 2 columns]


In [4]:
#counts of linkers 
linker_list = list(set(labelled['linker SMILES'].tolist()))
print(len(linker_list))

#counts of metals 
metal_df = pd.read_csv('./1M1L3D_metal_descriptors.csv')
metal_df = metal_df.fillna(0)
metal_df['Symbol'] = metal_df['Symbol'].str.strip()
metal_df.drop(['refcode'], axis=1, inplace=True)

metal_df.drop_duplicates(subset=['Symbol'],keep='first',inplace=True)
metal_df.set_index(["Symbol"],inplace=True)
print(len(metal_df))

3169
53


In [5]:
#creat unlabelled dataset
labelled = labelled.values.tolist()
unlabelled = [[metal,linker] for metal in metal_df.index.tolist() for linker in linker_list if [metal,linker] not in labelled]
unlabelled = pd.DataFrame(unlabelled)
unlabelled

,0,1
0,Co,O=[N+]([O-])Nc1nnn[nH]1
1,Co,O=C(O)c1ccc(-c2cccc(-c3ccc(C(=O)O)cc3)c2)cc1
2,Co,O=C(O)CC(CC(=O)O)C(=O)O
3,Co,Cc1nc(C(=O)O)c(C)nc1C(=O)O
4,Co,O=C(Nc1ccc(C(=O)O)c(C(=O)O)c1)C(=O)Nc1ccc(C(=O...
...,...,...
160577,W,O=C(O)c1ccc(-c2ccc(-c3cc(-c4ccc(-c5ccc(C(=O)O)...
160578,W,O=C(O)c1cc(C#Cc2cc(C(=O)O)cc(C(=O)O)c2)cc(C(=O...
160579,W,CCc1cnc(C2=NC(=O)[C@](C)(C(C)C)N2)c(C(=O)O)c1
160580,W,c1ccc2c(-n3cncn3)c3ccccc3c(-n3cncn3)c2c1


In [6]:
#output the ground_truth and the query datasets
labelled = pd.DataFrame(labelled)
labelled.columns=['metal','linker_SMILES']
unlabelled.columns=['metal','linker_SMILES']

labelled.to_csv('./ground_truth_pairs.csv')
unlabelled.to_csv('./query_pairs.csv')

In [8]:
#output linker list
import pickle
with open('linker_list.pkl','wb') as f:
    pickle.dump(linker_list,f)

# Featurization

### 6 elemental metal features

In [12]:
#normalization 
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

minmax_scaler = MinMaxScaler()
metal_df = metal_df.astype(float).fillna(0)
metal_MinMax_6 = pd.DataFrame(minmax_scaler.fit_transform(metal_df), columns = metal_df.columns, index = metal_df.index)

print(metal_MinMax_6)

metal_MinMax_6.to_csv('metal_scaled_6.csv')

        Atomic_Number  Atomic_Weight  Atomic Radius  Mulliken EN  \
Symbol                                                             
Co           0.269663       0.224989       0.215054     0.790378   
Ag           0.494382       0.436748       0.284946     0.838488   
Mn           0.247191       0.207700       0.263441     0.591065   
Tb           0.696629       0.657691       0.607527     0.378007   
Cu           0.292135       0.244950       0.177419     0.852234   
Fe           0.258427       0.211634       0.236559     0.707904   
Zn           0.303371       0.252930       0.161290     0.841924   
Pb           0.887640       0.866592       0.225806     0.652921   
La           0.606742       0.571058       0.446237     0.378007   
Eu           0.674157       0.627571       0.639785     0.378007   
Er           0.730337       0.693758       0.612903     0.378007   
Ni           0.280899       0.223951       0.198925     0.824742   
Cd           0.505618       0.456406       0.263

### 6 elemental metal features + 21-dimensional elemental features from magpie

In [13]:
#import 6 elemental metal features
metal_df = pd.read_csv('./1M1L3D_metal_descriptors.csv')
metal_df = metal_df.fillna(0)
metal_df['Symbol'] = metal_df['Symbol'].str.strip()
metal_df.drop(['refcode'], axis=1, inplace=True)

metal_df.drop_duplicates(subset=['Symbol'],keep='first',inplace=True)
metal_df.set_index(["Symbol"],inplace=True)

#add metal features from magpie
from ElMD import ElMD
featurizingDict = ElMD(metric="magpie").periodic_tab

magpie_len = len(featurizingDict["H"])
x = np.arange(1,magpie_len + 1,1)
colume_name = [str("magpie") + str(i) for i in x]

magpie_df = pd.DataFrame(index=metal_df.index,columns=colume_name)
for i in metal_df.index:
    magpie_df.loc[i] = list(featurizingDict[i])
metal_df = pd.concat([metal_df,magpie_df],axis=1)
metal_df = metal_df.fillna(0)

#normalization
minmax_scaler = MinMaxScaler()
metal_df = metal_df.astype(float).fillna(0)
metal_MinMax_27 = pd.DataFrame(minmax_scaler.fit_transform(metal_df), columns = metal_df.columns, index = metal_df.index)

print(metal_MinMax_27)

        Atomic_Number  Atomic_Weight  Atomic Radius  Mulliken EN  \
Symbol                                                             
Co           0.269663       0.224989       0.215054     0.790378   
Ag           0.494382       0.436748       0.284946     0.838488   
Mn           0.247191       0.207700       0.263441     0.591065   
Tb           0.696629       0.657691       0.607527     0.378007   
Cu           0.292135       0.244950       0.177419     0.852234   
Fe           0.258427       0.211634       0.236559     0.707904   
Zn           0.303371       0.252930       0.161290     0.841924   
Pb           0.887640       0.866592       0.225806     0.652921   
La           0.606742       0.571058       0.446237     0.378007   
Eu           0.674157       0.627571       0.639785     0.378007   
Er           0.730337       0.693758       0.612903     0.378007   
Ni           0.280899       0.223951       0.198925     0.824742   
Cd           0.505618       0.456406       0.263

In [14]:
metal_MinMax_27.to_csv('metal_scaled_27.csv')

### 6 elemental metal features + 199-dimensional NLP features from mat2vec

In [16]:
#import 6 elemental metal features
metal_df = pd.read_csv('./1M1L3D_metal_descriptors.csv')
metal_df = metal_df.fillna(0)
metal_df['Symbol'] = metal_df['Symbol'].str.strip()
metal_df.drop(['refcode'], axis=1, inplace=True)

metal_df.drop_duplicates(subset=['Symbol'],keep='first',inplace=True)
metal_df.set_index(["Symbol"],inplace=True)

#add metal features from mat2vec
from ElMD import ElMD
featurizingDict = ElMD(metric="mat2vec").periodic_tab

mat2vec_len = len(featurizingDict["H"])
x = np.arange(1,mat2vec_len + 1,1)
colume_name = [str("mat2vec") + str(i) for i in x]

mat2vec_df = pd.DataFrame(index=metal_df.index,columns=colume_name)
for i in metal_df.index:
    mat2vec_df.loc[i] = list(featurizingDict[i])
metal_df = pd.concat([metal_df,mat2vec_df],axis=1)
metal_df = metal_df.fillna(0)

#normalization
minmax_scaler = MinMaxScaler()
metal_df = metal_df.astype(float).fillna(0)
metal_MinMax_205 = pd.DataFrame(minmax_scaler.fit_transform(metal_df), columns = metal_df.columns, index = metal_df.index)

print(metal_MinMax_205)
metal_MinMax_205.to_csv('metal_scaled_205.csv')

        Atomic_Number  Atomic_Weight  Atomic Radius  Mulliken EN  \
Symbol                                                             
Co           0.269663       0.224989       0.215054     0.790378   
Ag           0.494382       0.436748       0.284946     0.838488   
Mn           0.247191       0.207700       0.263441     0.591065   
Tb           0.696629       0.657691       0.607527     0.378007   
Cu           0.292135       0.244950       0.177419     0.852234   
Fe           0.258427       0.211634       0.236559     0.707904   
Zn           0.303371       0.252930       0.161290     0.841924   
Pb           0.887640       0.866592       0.225806     0.652921   
La           0.606742       0.571058       0.446237     0.378007   
Eu           0.674157       0.627571       0.639785     0.378007   
Er           0.730337       0.693758       0.612903     0.378007   
Ni           0.280899       0.223951       0.198925     0.824742   
Cd           0.505618       0.456406       0.263

### 1613 Mordred linker features 

In [9]:
#import linker list
import pickle
with open('linker_list.pkl','rb') as f:
    linker_list = pickle.load(f)
print(len(linker_list))

3169


In [10]:
#create Mordred descriptor calculator with all descriptors
calc = Calculator(descriptors, ignore_3D=True)
linker_df = pd.DataFrame()

#create Mordred descriptors for linkers
for linker in linker_list:
    mol = Chem.MolFromSmiles(linker)  # calculate single molecule
    results = calc(mol).asdict()
    
    for key in results:
        if type(results[key]) == mordred.error.Missing:
            results[key] = None
            
    result_pd = pd.DataFrame(results, index = [linker])    
    linker_df = linker_df.append(result_pd)

print(linker_df.isnull().any().sum())
linker_df = linker_df.fillna(0)
linker_df = linker_df.astype(float).fillna(0)

C:\ANACONDA3\2021.05\envs\py37\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


516


In [11]:
linker_df

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
O=[N+]([O-])Nc1nnn[nH]1,6.582741,6.621580,2.0,0.0,10.764721,2.190708,4.242061,10.764721,1.196080,3.093038,...,8.356790,49.928003,130.023923,11.820357,93.0,6.0,40.0,42.0,3.472222,2.083333
O=C(O)c1ccc(-c2cccc(-c3ccc(C(=O)O)cc3)c2)cc1,18.660575,14.062695,2.0,0.0,31.154444,2.384292,4.768585,31.154444,1.298102,4.102984,...,10.013821,58.446935,318.089209,8.370769,1487.0,37.0,124.0,144.0,7.888889,5.277778
O=C(O)CC(CC(=O)O)C(=O)O,8.394073,8.664551,3.0,0.0,12.654059,2.193993,4.387987,12.654059,1.054505,3.321971,...,8.676417,41.105781,176.032088,8.801604,211.0,12.0,50.0,51.0,6.944444,2.777778
Cc1nc(C(=O)O)c(C)nc1C(=O)O,10.394073,9.939889,2.0,0.0,16.341834,2.379878,4.759756,16.341834,1.167274,3.533817,...,9.484101,45.572577,196.048407,8.911291,291.0,21.0,68.0,78.0,7.166667,3.111111
O=C(Nc1ccc(C(=O)O)c(C(=O)O)c1)C(=O)Nc1ccc(C(=O)O)c(C(=O)O)c1,22.731127,18.805627,4.0,0.0,35.938872,2.386888,4.773776,35.938872,1.197962,4.298100,...,10.227490,65.418121,416.049195,9.905933,2757.0,48.0,150.0,173.0,13.333333,6.611111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
O=C(O)c1ccc(-c2ccc(-c3cc(-c4ccc(-c5ccc(C(=O)O)cc5)s4)cc(-c4ccc(-c5ccc(C(=O)O)cc5)s4)c3)s2)cc1,38.476144,26.445299,3.0,0.0,63.399614,2.475393,4.866350,63.399614,1.320825,4.818337,...,10.811323,103.110073,684.073501,9.501021,9672.0,78.0,264.0,315.0,14.000000,10.250000
O=C(O)c1cc(C#Cc2cc(C(=O)O)cc(C(=O)O)c2)cc(C(=O)O)c1,19.805241,16.492157,4.0,0.0,31.631891,2.369521,4.739041,31.631891,1.216611,4.158637,...,10.031661,60.608662,354.037567,9.834377,1767.0,39.0,130.0,148.0,11.111111,5.694444
CCc1cnc(C2=NC(=O)[C@](C)(C(C)C)N2)c(C(=O)O)c1,16.017677,14.515122,1.0,2.0,25.609688,2.503131,4.882831,25.609688,1.219509,3.965373,...,10.098437,69.500414,289.142641,7.228566,885.0,36.0,110.0,132.0,9.340278,4.625000
c1ccc2c(-n3cncn3)c3ccccc3c(-n3cncn3)c2c1,19.475469,15.887959,0.0,0.0,33.151826,2.565579,5.115723,33.151826,1.381326,4.155375,...,10.352555,74.480952,312.112344,8.669787,1132.0,41.0,136.0,168.0,4.888889,5.222222


In [17]:
#normalization
linker_MinMax_1613 = pd.DataFrame(minmax_scaler.fit_transform(linker_df), columns = linker_df.columns, index = linker_df.index)
print(linker_MinMax_1613)
linker_MinMax_1613.to_csv('linker_scaled_1613.csv')

                                                         ABC     ABCGG  \
O=[N+]([O-])Nc1nnn[nH]1                             0.068014  0.110007   
O=C(O)c1ccc(-c2cccc(-c3ccc(C(=O)O)cc3)c2)cc1        0.192806  0.233629   
O=C(O)CC(CC(=O)O)C(=O)O                             0.086730  0.143948   
Cc1nc(C(=O)O)c(C)nc1C(=O)O                          0.107394  0.165135   
O=C(Nc1ccc(C(=O)O)c(C(=O)O)c1)C(=O)Nc1ccc(C(=O)...  0.234863  0.312425   
...                                                      ...       ...   
O=C(O)c1ccc(-c2ccc(-c3cc(-c4ccc(-c5ccc(C(=O)O)c...  0.397545  0.439346   
O=C(O)c1cc(C#Cc2cc(C(=O)O)cc(C(=O)O)c2)cc(C(=O)...  0.204632  0.273991   
CCc1cnc(C2=NC(=O)[C@](C)(C(C)C)N2)c(C(=O)O)c1       0.165498  0.241145   
c1ccc2c(-n3cncn3)c3ccccc3c(-n3cncn3)c2c1            0.201225  0.263953   
[C][C@@H](CC)CC1(C[C@@H](C)CC)c2cc(-c3cc(C(=O)O...  0.378875  0.468851   

                                                       nAcid     nBase  \
O=[N+]([O-])Nc1nnn[nH]1              

### 177 Mordred linker features

In [19]:
linker_unscaled_fe = linker_df

#linker feature engineering
#drop the highly linearly correlated features among the datasets
#create correlation matrix
corr_matrix1 = linker_unscaled_fe.corr().abs()
#select upper triangle of correlation matrix
upper1 = corr_matrix1.where(np.triu(np.ones(corr_matrix1.shape), k=1).astype(np.bool))

#find index of feature columns with Pearson correlation greater than 0.80
to_drop1 = [column for column in upper1.columns if any(upper1[column] > 0.80)]
#drop the descriptos with low variance, below 0.5
drop = linker_unscaled_fe.std()[linker_unscaled_fe.std() < 0.5].index.values
to_drop2= [x for x in drop if x not in to_drop1]
drop_final= to_drop1 + to_drop2

C:\ANACONDA3\2021.05\envs\py37\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [21]:
#drop columns with high correlation and low variance
linker_MinMax_177 = linker_MinMax_1613.drop(columns=drop_final)
#output
linker_MinMax_177.to_csv('linker_scaled_177.csv')

In [22]:
linker_MinMax_177

,ABC,nAcid,nBase,VR1_A,nBridgehead,nHetero,nN,nO,nS,nF,...,MDEC-12,MDEC-14,MDEC-22,MDEC-23,MDEC-33,MDEN-22,MDEN-23,nHRing,n6HRing,nARing
O=[N+]([O-])Nc1nnn[nH]1,0.068014,0.166667,0.000000,0.000152,0.0,0.133333,0.222222,0.040816,0.000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.177838,0.141819,0.076923,0.000,0.000000
O=C(O)c1ccc(-c2cccc(-c3ccc(C(=O)O)cc3)c2)cc1,0.192806,0.166667,0.000000,0.000852,0.0,0.066667,0.000000,0.081633,0.000,0.0,...,0.000000,0.000000,0.281436,0.291668,0.081998,0.000000,0.000000,0.000000,0.000,0.000000
O=C(O)CC(CC(=O)O)C(=O)O,0.086730,0.250000,0.000000,0.000216,0.0,0.100000,0.000000,0.122449,0.000,0.0,...,0.000000,0.000000,0.007950,0.054789,0.032425,0.000000,0.000000,0.000000,0.000,0.000000
Cc1nc(C(=O)O)c(C)nc1C(=O)O,0.107394,0.166667,0.000000,0.000296,0.0,0.100000,0.074074,0.081633,0.000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.085934,0.010443,0.000000,0.076923,0.125,0.000000
O=C(Nc1ccc(C(=O)O)c(C(=O)O)c1)C(=O)Nc1ccc(C(=O)O)c(C(=O)O)c1,0.234863,0.333333,0.000000,0.000977,0.0,0.200000,0.074074,0.204082,0.000,0.0,...,0.000000,0.000000,0.055457,0.191436,0.177593,0.010443,0.000000,0.000000,0.000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
O=C(O)c1ccc(-c2ccc(-c3cc(-c4ccc(-c5ccc(C(=O)O)cc5)s4)cc(-c4ccc(-c5ccc(C(=O)O)cc5)s4)c3)s2)cc1,0.397545,0.250000,0.000000,0.004158,0.0,0.150000,0.000000,0.122449,0.375,0.0,...,0.000000,0.000000,0.501635,0.675013,0.293365,0.000000,0.000000,0.230769,0.000,0.000000
O=C(O)c1cc(C#Cc2cc(C(=O)O)cc(C(=O)O)c2)cc(C(=O)O)c1,0.204632,0.333333,0.000000,0.000815,0.0,0.133333,0.000000,0.163265,0.000,0.0,...,0.000000,0.000000,0.133449,0.254659,0.135380,0.000000,0.000000,0.000000,0.000,0.000000
CCc1cnc(C2=NC(=O)[C@](C)(C(C)C)N2)c(C(=O)O)c1,0.165498,0.083333,0.166667,0.000762,0.0,0.100000,0.111111,0.061224,0.000,0.0,...,0.068356,0.137135,0.023851,0.075034,0.091721,0.035864,0.000000,0.153846,0.125,0.083333
c1ccc2c(-n3cncn3)c3ccccc3c(-n3cncn3)c2c1,0.201225,0.000000,0.000000,0.001000,0.0,0.100000,0.222222,0.000000,0.000,0.0,...,0.000000,0.000000,0.245528,0.249587,0.112904,0.037398,0.202515,0.153846,0.000,0.000000


### RDkit features 

In [23]:
calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
desc_names = calc.GetDescriptorNames()
desc_names

('MaxEStateIndex',
 'MinEStateIndex',
 'MaxAbsEStateIndex',
 'MinAbsEStateIndex',
 'qed',
 'MolWt',
 'HeavyAtomMolWt',
 'ExactMolWt',
 'NumValenceElectrons',
 'NumRadicalElectrons',
 'MaxPartialCharge',
 'MinPartialCharge',
 'MaxAbsPartialCharge',
 'MinAbsPartialCharge',
 'FpDensityMorgan1',
 'FpDensityMorgan2',
 'FpDensityMorgan3',
 'BCUT2D_MWHI',
 'BCUT2D_MWLOW',
 'BCUT2D_CHGHI',
 'BCUT2D_CHGLO',
 'BCUT2D_LOGPHI',
 'BCUT2D_LOGPLOW',
 'BCUT2D_MRHI',
 'BCUT2D_MRLOW',
 'BalabanJ',
 'BertzCT',
 'Chi0',
 'Chi0n',
 'Chi0v',
 'Chi1',
 'Chi1n',
 'Chi1v',
 'Chi2n',
 'Chi2v',
 'Chi3n',
 'Chi3v',
 'Chi4n',
 'Chi4v',
 'HallKierAlpha',
 'Ipc',
 'Kappa1',
 'Kappa2',
 'Kappa3',
 'LabuteASA',
 'PEOE_VSA1',
 'PEOE_VSA10',
 'PEOE_VSA11',
 'PEOE_VSA12',
 'PEOE_VSA13',
 'PEOE_VSA14',
 'PEOE_VSA2',
 'PEOE_VSA3',
 'PEOE_VSA4',
 'PEOE_VSA5',
 'PEOE_VSA6',
 'PEOE_VSA7',
 'PEOE_VSA8',
 'PEOE_VSA9',
 'SMR_VSA1',
 'SMR_VSA10',
 'SMR_VSA2',
 'SMR_VSA3',
 'SMR_VSA4',
 'SMR_VSA5',
 'SMR_VSA6',
 'SMR_VSA7',
 'SMR_

In [24]:
def RDkit_descriptors(smiles):
    mols = [Chem.MolFromSmiles(i) for i in smiles]
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()
    
    Mol_descriptors = []
    for mol in mols:
        # add hydrogens to molecules
        mol=Chem.AddHs(mol)
        # calculate all 208 descriptors for each molecule
        descriptors = calc.CalcDescriptors(mol)
        Mol_descriptors.append(descriptors)
    return Mol_descriptors, desc_names

Mol_descriptors, desc_names = RDkit_descriptors(linker_list)
linker_df = pd.DataFrame(Mol_descriptors, index=linker_list, columns=desc_names)
linker_df

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
O=[N+]([O-])Nc1nnn[nH]1,9.901667,-1.058796,9.901667,0.201389,0.384122,130.067,128.051,130.023923,48,0,...,0,0,0,0,1,0,0,0,0,0
O=C(O)c1ccc(-c2cccc(-c3ccc(C(=O)O)cc3)c2)cc1,11.847995,-1.510128,11.847995,0.739871,0.746602,318.328,304.216,318.089209,118,0,...,0,0,0,0,0,0,0,0,0,0
O=C(O)CC(CC(=O)O)C(=O)O,11.352605,-3.974190,11.352605,2.180967,0.526728,176.124,168.060,176.032088,68,0,...,0,0,0,0,0,0,0,0,0,0
Cc1nc(C(=O)O)c(C)nc1C(=O)O,11.411690,-3.054710,11.411690,1.039208,0.713371,196.162,188.098,196.048407,74,0,...,0,0,0,0,0,0,0,0,0,0
O=C(Nc1ccc(C(=O)O)c(C(=O)O)c1)C(=O)Nc1ccc(C(=O)O)c(C(=O)O)c1,12.838263,-2.197220,12.838263,0.608368,0.368810,416.298,404.202,416.049195,154,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
O=C(O)c1ccc(-c2ccc(-c3cc(-c4ccc(-c5ccc(C(=O)O)cc5)s4)cc(-c4ccc(-c5ccc(C(=O)O)cc5)s4)c3)s2)cc1,12.293802,-1.554015,12.293802,0.362205,0.139499,684.816,660.624,684.073501,234,0,...,0,0,0,0,0,0,0,3,0,0
O=C(O)c1cc(C#Cc2cc(C(=O)O)cc(C(=O)O)c2)cc(C(=O)O)c1,11.929863,-1.602819,11.929863,0.810383,0.607972,354.270,344.190,354.037567,130,0,...,0,0,0,0,0,0,0,0,0,0
CCc1cnc(C2=NC(=O)[C@](C)(C(C)C)N2)c(C(=O)O)c1,13.216993,-4.314753,13.216993,0.616977,0.878698,289.335,270.183,289.142641,112,0,...,0,0,0,0,0,0,0,0,0,0
c1ccc2c(-n3cncn3)c3ccccc3c(-n3cncn3)c2c1,8.624899,-0.677318,8.624899,0.311703,0.469773,312.336,300.240,312.112344,114,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
#drop feature columns with all zeros
linker_df = linker_df.loc[:,~(linker_df==0).all(axis=0)]
print(linker_df.shape)
print(linker_df.isnull().sum().sum())
linker_df = linker_df.fillna(0)

#normalization
minmax_scaler = MinMaxScaler()
linker_MinMax = pd.DataFrame(minmax_scaler.fit_transform(linker_df), columns = linker_df.columns, index = linker_df.index)
linker_MinMax

(3169, 194)
240


,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_urea
O=[N+]([O-])Nc1nnn[nH]1,0.484727,0.819426,0.484727,0.020356,0.415246,0.063506,0.067363,0.063536,0.059859,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0
O=C(O)c1ccc(-c2cccc(-c3ccc(C(=O)O)cc3)c2)cc1,0.610185,0.781626,0.610185,0.074836,0.819599,0.185471,0.185985,0.185467,0.183099,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
O=C(O)CC(CC(=O)O)C(=O)O,0.578253,0.575252,0.578253,0.220636,0.574325,0.093344,0.094303,0.093365,0.095070,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
Cc1nc(C(=O)O)c(C)nc1C(=O)O,0.582061,0.652262,0.582061,0.105121,0.782530,0.106325,0.107796,0.106343,0.105634,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
O=C(Nc1ccc(C(=O)O)c(C(=O)O)c1)C(=O)Nc1ccc(C(=O)O)c(C(=O)O)c1,0.674016,0.724079,0.674016,0.061532,0.398165,0.248940,0.253311,0.248979,0.246479,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
O=C(O)c1ccc(-c2ccc(-c3cc(-c4ccc(-c5ccc(C(=O)O)cc5)s4)cc(-c4ccc(-c5ccc(C(=O)O)cc5)s4)c3)s2)cc1,0.638921,0.777950,0.638921,0.036627,0.142364,0.422900,0.425973,0.422751,0.387324,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.0
O=C(O)c1cc(C#Cc2cc(C(=O)O)cc(C(=O)O)c2)cc(C(=O)O)c1,0.615462,0.773862,0.615462,0.081970,0.664955,0.208756,0.212901,0.208774,0.204225,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
CCc1cnc(C2=NC(=O)[C@](C)(C(C)C)N2)c(C(=O)O)c1,0.698428,0.546729,0.698428,0.062403,0.966954,0.166687,0.163068,0.166700,0.172535,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
c1ccc2c(-n3cncn3)c3ccccc3c(-n3cncn3)c2c1,0.402429,0.851376,0.402429,0.031517,0.510791,0.181589,0.183307,0.181592,0.176056,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [26]:
linker_MinMax.to_csv('linker_scaled_rdkit.csv')